In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression

import xgboost as xgb

import seaborn as sns
sns.set(context='paper', style='darkgrid', rc={'figure.facecolor':'white'}, font_scale=1.2)

import sqlite3

In [2]:
df = pd.read_csv('train.csv', index_col = 0)

In [6]:
df.columns

Index(['bone_length', 'rotting_flesh', 'hair_length', 'has_soul', 'color',
       'type'],
      dtype='object')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 371 entries, 0 to 897
Data columns (total 6 columns):
bone_length      371 non-null float64
rotting_flesh    371 non-null float64
hair_length      371 non-null float64
has_soul         371 non-null float64
color            371 non-null object
type             371 non-null object
dtypes: float64(4), object(2)
memory usage: 20.3+ KB


# Practicing Pandas

In [22]:
df[df.type == 'Ghost'][['hair_length', 'has_soul']].sort_values('hair_length', ascending = False)

,hair_length,has_soul
id,,
654,0.636340,0.248143
809,0.629068,0.292060
804,0.609449,0.384293
340,0.597167,0.292293
359,0.587628,0.237966
...,...,...
616,0.164321,0.300659
507,0.164074,0.594088
465,0.162715,0.099904


In [33]:
df.groupby(['type', 'color']).count()

bone_length  rotting_flesh  hair_length  has_soul
type   color                                                   
Ghost  black           14             14           14        14
       blood            6              6            6         6
       blue             6              6            6         6
       clear           32             32           32        32
       green           15             15           15        15
       white           44             44           44        44
Ghoul  black           14             14           14        14
       blood            4              4            4         4
       blue             6              6            6         6
       clear           42             42           42        42
       green           13             13           13        13
       white           50             50           50        50
Goblin black           13             13           13        13
       blood            2              2            2         2
       blue             7              7            7         7
       clear           46             46           46        46
       green           14             14           14        14
       white           43             43           43        43

# Practicing SQL problems

In [7]:
# connect to the csv file and create a database
conn = sqlite3.connect('train.db')

# connect the cursor to be able to communicate with the database through the code
c = conn.cursor()

In [9]:
# creating a dataframe
c.execute("""CREATE TABLE train (
            bone_length float, 
            rotting_flesh float, 
            hair_length float, 
            has_soul float, 
            color text,
            type text 
            )""")

conn.commit()

In [10]:
# add data from the datafram to the sql database

df.to_sql('train', conn, if_exists = 'append', index = False)

In [11]:
c.execute('SELECT * from train').fetchall()

[(0.3545121845821541,
  0.35083902671065004,
  0.4657608918291205,
  0.78114166586219,
  'clear',
  'Ghoul'),
 (0.5755599050254655,
  0.42586843221058457,
  0.5314013787091408,
  0.43989887703728897,
  'green',
  'Goblin'),
 (0.467875498742323,
  0.3543304203917748,
  0.8116160896689244,
  0.7912249733377691,
  'black',
  'Ghoul'),
 (0.7766524606554741,
  0.5087225499432287,
  0.6367655799597881,
  0.8844636920976973,
  'black',
  'Ghoul'),
 (0.56611660205779,
  0.8758617955062562,
  0.4185936708748797,
  0.6364378187279373,
  'green',
  'Ghost'),
 (0.40567974489188297,
  0.253277496666039,
  0.4414196710726873,
  0.2803238199213896,
  'green',
  'Goblin'),
 (0.39933085086043796,
  0.5689517671030597,
  0.6183910202674578,
  0.4679008345210921,
  'white',
  'Goblin'),
 (0.5162238981116547,
  0.5364287456984622,
  0.6127761465851604,
  0.4680482696484728,
  'clear',
  'Ghoul'),
 (0.31429527592019163,
  0.6712796900808566,
  0.4172669166262837,
  0.2275475750972251,
  'blue',
  'Ghost'),

In [14]:
c.execute("SELECT AVG(bone_length) FROM train WHERE type = 'Goblin'").fetchall()

[(0.42965028016685447,)]

In [17]:
c.execute('SELECT has_soul, hair_length FROM train WHERE type = "Ghost" ORDER BY hair_length DESC LIMIT 5').fetchall()



[(0.2481432621832249, 0.6363397309473561),
 (0.29205988696696245, 0.6290679358102901),
 (0.3842929461674824, 0.609449079243181),
 (0.29229288861696906, 0.5971668096599541),
 (0.2379662800942501, 0.5876280121451709)]

In [23]:
c.execute('SELECT min(has_soul), max(has_soul), type FROM train GROUP BY type').fetchall()

[(0.009401615878661942, 0.6899803696223118, 'Ghost'),
 (0.2808536942554318, 0.9357213331107028, 'Ghoul'),
 (0.19441331044345034, 0.8549489712660003, 'Goblin')]

In [29]:
c.execute('SELECT count(type), color, type FROM train GROUP BY type, color').fetchall()

[(14, 'black', 'Ghost'),
 (6, 'blood', 'Ghost'),
 (6, 'blue', 'Ghost'),
 (32, 'clear', 'Ghost'),
 (15, 'green', 'Ghost'),
 (44, 'white', 'Ghost'),
 (14, 'black', 'Ghoul'),
 (4, 'blood', 'Ghoul'),
 (6, 'blue', 'Ghoul'),
 (42, 'clear', 'Ghoul'),
 (13, 'green', 'Ghoul'),
 (50, 'white', 'Ghoul'),
 (13, 'black', 'Goblin'),
 (2, 'blood', 'Goblin'),
 (7, 'blue', 'Goblin'),
 (46, 'clear', 'Goblin'),
 (14, 'green', 'Goblin'),
 (43, 'white', 'Goblin')]